In [1]:
import readline
import rpy2.robjects
import code.independence_test as it
import code.additive_noise as an
import numpy as np
import numpy.random as rn
import code.hsic as hs
import pandas as pd

In [ ]:
n=200
uni = rn.uniform(0,10,n)
exp = rn.exponential(7,n)
gamma = rn.gamma(10,1,n)
log = rn.logistic(10,0.5,n)
n_Y = np.random.uniform(0,0.01,n)
n_L = np.random.uniform(0,0.01,n)
#Y = uni + exp + gamma + log + n_Y
Y = gamma + n_Y
leak = Y + n_L

In [ ]:
X = np.stack([gamma, leak, log],axis=1)

In [ ]:
X = pd.DataFrame(X, columns=['cause', 'leak', 'independent'])

In [ ]:
an.ANM_algorithm(X, Y, 'py')

In [30]:
NUM_EXAMPLES = 200
SIZES = [200] * NUM_EXAMPLES + [300] * NUM_EXAMPLES + [500] * NUM_EXAMPLES + [1000] * NUM_EXAMPLES

In [31]:
# Distributions
D = {
     'uniform': lambda: rn.uniform(-1, 1, SIZE)
}

# Function transformations
non_gaussian_noise = lambda: rn.uniform(-0.01, 0.01, SIZE)
gaussian_noise = lambda: rn.normal(0, 0.001, SIZE)

F = {
    'identity': lambda x: x,
    'scale': lambda x: rn.randint(1, 10) * rn.randn(1) * x,
    'exp': lambda x: np.exp(x.astype('float')),
    'square': lambda x: x ** 2,
    'cube': lambda x: x ** 3,
}

def add(a, b):
    if a is None:
        a = np.zeros_like(a)
    return a + b

def sub(a, b):
    if a is None:
        a = np.zeros_like(a)
    return a - b

def mul(a, b):
    if a is None:
        a = np.ones_like(a)
    return a * b

def div(a, b):
    if a is None:
        a = np.ones_like(a)
    return a / b

# Possible combinations of features
C = {
    '+': add,
    '-': sub,
    '*': mul,
    '/': div
}

In [32]:
all_tests = pd.DataFrame()
tests_features = []
tests_ys = []

In [33]:
for iteration, SIZE in enumerate(SIZES):
    # Total numbr of features
    N = rn.randint(1, 5)

    # Selected to form Y
    if N > 1:
        S = rn.randint(1, N)
    else:
        S = 1

    L = 1

    print N, S, L

    # Generate Features
    all_features = {}

    for i in range(N):
        dist = rn.randint(0, len(D))
        feature = {}
        feature['dist'] = D.keys()[dist]
        feature['values'] = D.values()[dist]()
        all_features['X_{}'.format(i)] = feature

    # Apply transformations to features
    candidates = list(range(N))
    rn.shuffle(candidates)

    for i in candidates[:S]:
        func = rn.randint(0, len(F))
        feature = all_features['X_{}'.format(i)]
        feature['func'] = F.keys()[func]
        feature['transformed'] = F.values()[func](feature['values'])

    # Generate Y from selected features
    Y = {"comb":"Y = ",
         "values": None}

    for i in range(N):
        if 'transformed' in all_features.values()[i].keys():
            comb = rn.randint(0, len(C))
            Y['comb'] += "{} f({}) ".format(C.keys()[comb], all_features.keys()[i])
            Y['values'] = C.values()[comb](Y['values'], all_features.values()[i]['transformed'])
            
    # Add noise to Y
    Y['values'] += non_gaussian_noise()

    for i in range(L):
        feature = {}
        func = rn.randint(0, len(F))
        feature['func'] = F.keys()[func]
        feature['values'] = F.values()[func](Y['values'])
        # Add noise to leaked variable
        feature['values'] += non_gaussian_noise()
        all_features['leak_{}'.format(i)] = feature
    
    columns = [name for name in all_features.keys()]
    causes = [i for i, f in enumerate(all_features.values()) if f.get('transformed', np.array([])).shape[0] > 0]
    for j in range(len(columns)):
        if j in causes:
            columns[j] = 'cause_' + columns[j]
        elif not columns[j].startswith('leak'):
            columns[j] = 'independent_' + columns[j]

    tests_features.append(all_features)
    tests_ys.append(Y)

    final_data = np.stack([f['values'] for f in all_features.values()], axis=1).astype('float')
    data = pd.DataFrame(final_data, columns=columns)
    standarized = (data - data.mean()) / data.std()
    
    try:
        res = an.ANM_algorithm(standarized, Y['values'].astype('float'))
    except:
        continue
    
    res['Iteration'] = iteration
    res['Size'] = SIZE
    res['Vars'] = res.index
    res['Truth'] = [idx.split('_')[0] for idx in res.index]
    
    all_tests = all_tests.append(res, ignore_index=True)

3 2 1
Number of line searches 40
Number of line searches 40
Number of line searches 2
Number of line searches 2
Number of line searches 40
Number of line searches 40
Number of line searches 40
Number of line searches 38
4 1 1
Number of line searches 40
Number of line searches 40
Number of line searches 40
Number of line searches 40
Number of line searches 40
Number of line searches 40
Number of line searches 40
Number of line searches 40
Number of line searches 30
Number of line searches 40
2 1 1
Number of line searches 19
Number of line searches 12
Number of line searches 40
Number of line searches 40
Number of line searches 21
Number of line searches 21
1 1 1
Number of line searches 28
Number of line searches 16
Number of line searches 40
Number of line searches 40
3 2 1
Number of line searches 40
Number of line searches 16
Number of line searches 40
Number of line searches 40
Number of line searches 40
Number of line searches 39
Number of line searches 40
Number of line searches 40


In [34]:
all_tests

,Direction,Rank,P_value Independence X and Y,Vars,Truth
0,Leakage,1.09734e-19,1.13135e-283,leak_0_0,leak
1,Independent,-0.00012068,0.151643,independent_X_2_0,independent
2,No Leakage,-0.00159494,2.11024e-62,cause_X_1_0,cause
3,No Leakage,-0.00576176,2.46525e-45,cause_X_0_0,cause
4,Independent,4.43678e-08,0.317931,independent_X_3_1,independent
5,Leakage,3.85926e-12,0.00220046,independent_X_2_1,independent
6,No Leakage,-1.17917e-08,3.01675e-291,leak_0_1,leak
7,Independent,-1.99004e-07,0.303535,independent_X_0_1,independent
8,No Leakage,-4.90356e-07,6.0663e-185,cause_X_1_1,cause
9,Leakage,1.58785e-05,5.54502e-193,leak_0_2,leak


In [35]:
# OVERALL ACCURACY
all_truth = all_tests['Truth'] == 'leak'
accuracy_leak = ((all_tests['Direction'] == 'Leakage') & all_truth).sum() / float(all_truth.sum())

all_truth = all_tests['Truth'] == 'independent'
accuracy_independent = ((all_tests['Direction'] == 'Independent') & all_truth).sum() / float(all_truth.sum())

all_truth = (all_tests['Truth'] == 'cause')
accuracy_cause = ((all_tests['Direction'] == 'No Leakage') & all_truth).sum() / float(all_truth.sum())

accuracy_leak, accuracy_independent, accuracy_cause

In [36]:
accuracy_leak, accuracy_independent, accuracy_cause

(0.63736263736263732, 0.94782608695652171, 0.66393442622950816)

In [ ]:
# ACCURACY By DATASET  SIZE
for SIZE in set(SIZES):
    test_sub = all_tests.ix[all_tests['Size'] == SIZE]
    all_truth = all_tests['Truth'] == 'leak'
    accuracy_leak = ((all_tests['Direction'] == 'Leakage') & all_truth).sum() / float(all_truth.sum())

    all_truth = all_tests['Truth'] == 'independent'
    accuracy_independent = ((all_tests['Direction'] == 'Independent') & all_truth).sum() / float(all_truth.sum())

    all_truth = (all_tests['Truth'] == 'cause')
    accuracy_cause = ((all_tests['Direction'] == 'No Leakage') & all_truth).sum() / float(all_truth.sum())
    
    print 'SIZE', SIZE, accuracy_leak, accuracy_independent, accuracy_cause

# Since we know the groun truth, we can test that the errors are independent of the causing variable. This would tell us if the problem in estimation is with the regression or the independence test

In [ ]:
hsic = it.HSIC_b((Y['values'] - all_features['leak_0']['values']).astype('float'), Y['values'].astype('float'))

In [ ]:
hsic.empirical_test()

In [ ]:
hsic = it.HSIC_b((Y['values'] - all_features['X_0']['values']).astype('float'), Y['values'].astype('float'))

In [ ]:
hsic.empirical_test()

In [ ]:
non_gaussian_noise()